# FastPitch End-to-end Training

This notebook is designed to provide a guide on how to train FastPitch based end-to-end model as part of the TTS pipeline. It contains the following sections

  1. FastPitch End-to-end and NeMo - An introduction to the FastPitch End-to-end model
  2. LJSpeech - How to train FastPitch End-to-end on LJSpeech

# License

> Copyright 2023, NVIDIA CORPORATION & AFFILIATES. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL).
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator).
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect.
"""

# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget text-unidecode matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
# !python -m pip install "git+https://github.com/NVIDIA/NeMo.git@${BRANCH}#egg=nemo_toolkit[all]"

## Install pynini
# !wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/install_pynini.sh
# !bash install_pynini.sh

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

# FastPitch End-to-end and NeMo

FastPitch End-to-end is a joint model which combines FastPitch and HiFi-GAN to converts text characters directly into an audio sample. 

FastPitch End-to-end like most NeMo models are defined as a LightningModule, allowing for easy training via PyTorch Lightning, and parameterized by a configuration, currently defined via a yaml file and loading using Hydra.

In [ ]:
# Load the FastPitch End-to-end model
from nemo.collections.tts.models import FastPitchE2EModel
from nemo.collections.tts.models.base import TextToWaveform


TextToWaveform in NeMo have two helper functions:
   1. ```python
      parse(self, str_input: str),
      ``` 
      which takes an English string and produces a token tensor



   2. ```python
      convert_text_to_waveform(self, *, tokens),
      ```
      which takes the token tensor and generates an audio sample
Let's try it out

# Training

Let's see how to train a FastPitch End-to-end Model



In [ ]:
# NeMo's training scripts are stored inside the examples/ folder. Let's grab the fastpitch_e2e.py file
# as well as the fastpitch_e2e.yaml file
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/fastpitch_e2e.py
!(mkdir -p conf \
  && cd conf \
  && wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/fastpitch_e2e.yaml \
  && cd ..)

# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/ipa_cmudict-0.7b_nv23.01.txt \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/heteronyms-052722 \
&& cd ..
        

# Runner

Let's take a look at the fastpitch_e2e.py file

```python
import pytorch_lightning as pl

from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models.fastpitch_e2e import FastPitchE2EModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager
# hydra_runner is a thin NeMo wrapper around Hydra
# It looks for a config named vits.yaml inside the conf folder
# Hydra parses the yaml and returns it as a Omegaconf DictConfig
@hydra_runner(config_path="conf", config_name="fastpitch_e2e")
def main(cfg):
    # Define the Lightning trainer
    trainer = pl.Trainer(**cfg.trainer)
    # exp_manager is a NeMo construct that helps with logging and checkpointing
    exp_manager(trainer, cfg.get("exp_manager", None))
    # Define the VITS model, this will construct the model
    model = FastPitchE2EModel(cfg=cfg.model, trainer=trainer)
    # Let's add a few more callbacks
    trainer.callbacks.extend([pl.callbacks.LearningRateMonitor(), LogEpochTimeCallback()])
    # Call lightning trainer's fit() to train the model
    trainer.fit(model)

if __name__ == '__main__':
    main()  # noqa pylint: disable=no-value-for-parameter
```

# Config

Let's take a look at the yaml config

```yaml
name: &name FastPitch-E2E

train_dataset: ???
validation_datasets: ???
sup_data_types: ["align_prior_matrix", "pitch", "speaker_id"]
sup_data_types_val: ["align_prior_matrix", "pitch", "speaker_id"]

phoneme_dict_path: "scripts/tts_dataset_files/ipa_cmudict-0.7b_nv23.01.txt"
heteronyms_path: "scripts/tts_dataset_files/heteronyms-052722"
```

The first part of the yaml defines dataset parameters used by model. Then in the head of 'model' section there are processing - related parameters. You can see
that the sample rate is set to 44100 for HiFiTTS dataset. 

Looking at the yaml, there is `train_dataset: ???` and `validation_datasets: ???`. The ??? indicates to hydra that these values must be passed via the command line or the script will fail.


To train FastPitch-E2E model you need to either provide paths or names for FastPitch and HiFi-GAN models or provide corresponding configs:

```yaml
  fastpitch_model: null
  hifigan_model: null

  input_fft: #n_embed and padding_idx are added by the model
    _target_: nemo.collections.tts.modules.transformer.FFTransformerEncoder

  output_fft:
    _target_: nemo.collections.tts.modules.transformer.FFTransformerDecoder

  alignment_module:
    _target_: nemo.collections.tts.modules.aligner.AlignmentEncoder

  duration_predictor:
    _target_: nemo.collections.tts.modules.fastpitch.TemporalPredictor

  pitch_predictor:
    _target_: nemo.collections.tts.modules.fastpitch.TemporalPredictor

  generator:
    _target_: nemo.collections.tts.modules.hifigan_modules.Generator
```

Looking further down the yaml, we get to the pytorch lightning trainer parameters.

```yaml
trainer:
  num_nodes: 1
  devices: 2
  accelerator: gpu
  strategy: ddp
  precision: 32
  max_epochs: -1
  accumulate_grad_batches: 1
  enable_checkpointing: false # Provided by exp_manager
  logger: false # Provided by exp_manager
  log_every_n_steps: 50
  check_val_every_n_epoch: 1
```

These values can be changed either by editing the yaml or through the command line.

Let's grab some simple audio data and test VITS.

In [ ]:
!wget https://github.com/NVIDIA/NeMo/releases/download/v0.11.0/test_data.tar.gz \
&& mkdir -p tests/data \
&& tar xzf test_data.tar.gz -C tests/data

# FastPitch-E2E requires .json files to define the training and validation data.
!cat tests/data/asr/an4_val.json

Now that we have some sample data, we can try training FastPitch-E2E!

Note that the sample data is not enough data to fully train a FastPitch-E2E model. The following code uses a toy dataset to illustrate how the pipeline for training would work.

In [ ]:
!(python fastpitch_e2e.py \
  train_dataset=tests/data/asr/an4_train.json \
  validation_datasets=tests/data/asr/an4_val.json \
  sup_data_path=tests/data/sup_data \
  phoneme_dict_path=tts_dataset_files/ipa_cmudict-0.7b_nv23.01.txt \
  heteronyms_path=tts_dataset_files/heteronyms-052722 \
  model.fastpitch_model='tts_en_fastpitch_multispeaker' \
  model.hifigan_model='tts_en_hifitts_hifigan_ft_fastpitch' \
  trainer.max_epochs=3 \
  trainer.accelerator=null \
  trainer.check_val_every_n_epoch=1 \
  trainer.devices=1)

# Training Data

In order to train FastPitch-E2E, it is highly recommended to obtain high quality speech data with the following properties:
  - Sampling rate of 22050Hz or higher
  - Speech should contain a variety of speech phonemes
  - Audio split into segments of 1-10 seconds
  - Audio segments should not have silence at the beginning and end
  - Audio segments should not contain long silences inside

After obtaining the speech data and splitting into training, validation, and test sections, it is required to construct .json files to tell NeMo where to find these audio files.

The .json files should adhere to the format required by the `nemo.collections.tts.data.dataset.TTSDataset` class. For example, here is a sample .json file

```json
{"audio_filepath": "/path/to/audio1.wav", "text": "the transcription", "duration": 0.82}
{"audio_filepath": "/path/to/audio2.wav", "text": "the other transcription", "duration": 2.1}
...
```
Please note that the duration is in seconds.


## Evaluating FastPitch-E2E

Let's evaluate the quality of the FastPitch-E2E model.

FastPitch-E2E is end-to-end model, so we don't need any additional models to generate audios.

# TO-DO 
Add NGC checkpoint name for pretrained model for evaluation

In [ ]:
!wget https://multilangaudiosamples.s3.us-east-2.amazonaws.com/LJ023-0089.wav

In [ ]:
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import IPython.display as ipd
import numpy as np
import torch
import librosa
import soundfile as sf

target_sr = 44100

audio_path = "./LJ023-0089.wav"
text_raw = "That is not only my accusation."


audio_data, orig_sr = sf.read(audio_path)
if orig_sr != target_sr:
    audio_data = librosa.core.resample(audio_data, orig_sr=orig_sr, target_sr=target_sr)

# Let's double-check that everything matches up!
print(f"Duration (s): {len(audio_data)/target_sr}")
print("Transcript:", text_raw)
ipd.Audio(audio_data, rate=target_sr)

In [ ]:
model = FastPitchE2EModel.from_pretrained("").cpu().eval()

In [ ]:
tokens = model.parse(text_raw)
audio_pred = model.convert_text_to_waveform(tokens=tokens).cpu().detach().numpy()

print("predicted audio")
ipd.Audio(audio_pred, rate=target_sr)

In [ ]:
audio_to_mel = model.audio_to_melspec_processor


len_audio = torch.tensor(len(audio_data)).view(1, -1)

spec_pred, _ = audio_to_mel(torch.tensor(audio_pred).view(1, -1), len_audio)
spec_orig, _ = audio_to_mel(torch.tensor(audio_data).view(1, -1), len_audio)

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(spec_orig[0][0].cpu().detach())
ax[1].imshow(spec_pred[0][0].cpu().detach())

ax[0].set_title('Original spectrogram')
ax[1].set_title('Predicted spectrogram')
fig.show()